- load submission with iterative imp and reconstruct dataset with it - DONE

- for col in cols with missing values (or only F2_x and F4_x ?)
    - col is target
    - other cols are features
    - complete rows are for training
    - incomplete (with missing value) are for prediction
    - when the prediction is complete, update column values in dataset

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tqdm import tqdm

# Load Data

In [2]:
data = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv', index_col = 'row_id')
submission = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv', index_col = 'row-col')
sub_iter = pd.read_csv('../input/submission-iterativeimp/submission_iterative-imp.csv', index_col = 'row-col')

# display(data,submission,sub_iter)

In [3]:
# use results from iterative imputation as a base for next steps -> fill values into data
data_imp = data.copy()
for i in tqdm(sub_iter.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    data_imp.at[row,col] = sub_iter.loc[i]

100%|██████████| 1000000/1000000 [01:55<00:00, 8688.44it/s]


# Submission

In [4]:
for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i,'value'] = data_imp.loc[row,col]
    
    
submission.to_csv('submission.csv')

100%|██████████| 1000000/1000000 [01:42<00:00, 9735.56it/s]
